### Deliverable 1: Preprocessing the Data for a Neural Network

In [404]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [525]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [526]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME","ASK_AMT","INCOME_AMT"],inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,N,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,N,0
34297,T5,Independent,C3000,ProductDev,Association,1,N,1


In [527]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
STATUS                     2
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [528]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [529]:
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [530]:
# delete "Other" in AFFILIATION
application_df = application_df[(application_df["AFFILIATION"] != "Other") ]

In [531]:
application_df.USE_CASE.value_counts()

Preservation     28092
ProductDev        5670
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [532]:
# delete "Other" in USE_CASE
application_df = application_df[(application_df["USE_CASE"] != "Other") ]

In [533]:
application_df.ORGANIZATION.value_counts()

Trust           23511
Association     10254
Co-operative      486
Corporation        41
Name: ORGANIZATION, dtype: int64

In [534]:
application_df.INCOME_AMT.value_counts()

AttributeError: 'DataFrame' object has no attribute 'INCOME_AMT'

In [ ]:
application_df.SPECIAL_CONSIDERATIONS.value_counts()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_application = list(APPLICATION_TYPE_counts[APPLICATION_TYPE_counts < 100].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count<1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 90
hidden_nodes_layer3 = 70

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [535]:
# Compile the model

#keras.losses.SparseCategoricalCrossentropy()
#metrics=[keras.metrics.SparseCategoricalAccuracy()]
#"binary_crossentropy"
#metrics=["accuracy"]

import keras
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [536]:
# Train the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)

checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 99/804 [==>...........................] - ETA: 0s - loss: 0.6577 - accuracy: 0.6023  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
184/804 [=====>........................] - ETA: 0s - loss: 0.6611 - accuracy: 0.5948
Epoch 00001: saving model to checkpoints\weights.01.hdf5
271/804 [=========>....................] - ETA: 0s - loss: 0.6603 - accuracy: 0.5978
Epoch 00001: saving model to checkpoints\weights.01.hdf5
356/804 [============>.................] - ETA: 0s - loss: 0.6618 - accuracy: 0.5947
Epoch 00001: saving model to checkpoints\weights.01.hdf5
441/804 [===============>..............] - ETA: 0s - loss: 0.6626 - accuracy: 0.5912
Epoch 00001: saving model to checkpoints\weights.01.hdf5
525/804 [==================>...........] - ETA: 0s - loss: 0.6641 - accuracy: 0.5895
Epoch 00001: saving model to checkpoints\weights.01.hdf5
606/804 [=====================>........] - ETA: 0s - loss: 0.6627 - accuracy: 0.5924
Epoch 00001: saving model to checkpoints\weights.

571/804 [====================>.........] - ETA: 0s - loss: 0.6620 - accuracy: 0.5941
Epoch 00007: saving model to checkpoints\weights.07.hdf5
636/804 [======================>.......] - ETA: 0s - loss: 0.6630 - accuracy: 0.5929
Epoch 00007: saving model to checkpoints\weights.07.hdf5
771/804 [===========================>..] - ETA: 0s - loss: 0.6620 - accuracy: 0.5944
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 1s 731us/step - loss: 0.6619 - accuracy: 0.5951
Epoch 8/100
  1/804 [..............................] - ETA: 0s - loss: 0.6975 - accuracy: 0.5312
Epoch 00008: saving model to checkpoints\weights.08.hdf5
148/804 [====>.........................] - ETA: 0s - loss: 0.6624 - accuracy: 0.5887
Epoch 00008: saving model to checkpoints\weights.08.hdf5
207/804 [======>.......................] - ETA: 0s - loss: 0.6601 - accuracy: 0.5924
Epoch 00008: saving model to checkpoints\weights.08.hdf5
348/804 [===========>..................] - ET

138/804 [====>.........................] - ETA: 0s - loss: 0.6632 - accuracy: 0.5899
Epoch 00014: saving model to checkpoints\weights.14.hdf5
200/804 [======>.......................] - ETA: 0s - loss: 0.6614 - accuracy: 0.5944
Epoch 00014: saving model to checkpoints\weights.14.hdf5
329/804 [===========>..................] - ETA: 0s - loss: 0.6607 - accuracy: 0.5963
Epoch 00014: saving model to checkpoints\weights.14.hdf5
395/804 [=============>................] - ETA: 0s - loss: 0.6611 - accuracy: 0.5939
Epoch 00014: saving model to checkpoints\weights.14.hdf5
536/804 [===================>..........] - ETA: 0s - loss: 0.6608 - accuracy: 0.5950
Epoch 00014: saving model to checkpoints\weights.14.hdf5
602/804 [=====================>........] - ETA: 0s - loss: 0.6604 - accuracy: 0.5963
Epoch 00014: saving model to checkpoints\weights.14.hdf5
744/804 [==========================>...] - ETA: 0s - loss: 0.6620 - accuracy: 0.5945
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/80

609/804 [=====================>........] - ETA: 0s - loss: 0.6611 - accuracy: 0.5965
Epoch 00020: saving model to checkpoints\weights.20.hdf5
672/804 [========================>.....] - ETA: 0s - loss: 0.6612 - accuracy: 0.5964
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 1s 741us/step - loss: 0.6616 - accuracy: 0.5949
Epoch 21/100
  1/804 [..............................] - ETA: 0s - loss: 0.6541 - accuracy: 0.5938
Epoch 00021: saving model to checkpoints\weights.21.hdf5
 66/804 [=>............................] - ETA: 0s - loss: 0.6629 - accuracy: 0.5895
Epoch 00021: saving model to checkpoints\weights.21.hdf5
209/804 [======>.......................] - ETA: 0s - loss: 0.6612 - accuracy: 0.5952
Epoch 00021: saving model to checkpoints\weights.21.hdf5
275/804 [=========>....................] - ETA: 0s - loss: 0.6601 - accuracy: 0.5965
Epoch 00021: saving model to checkpoints\weights.21.hdf5
416/804 [==============>...............] - E

144/804 [====>.........................] - ETA: 0s - loss: 0.6575 - accuracy: 0.5983
Epoch 00027: saving model to checkpoints\weights.27.hdf5
284/804 [=========>....................] - ETA: 0s - loss: 0.6571 - accuracy: 0.6001
Epoch 00027: saving model to checkpoints\weights.27.hdf5
350/804 [============>.................] - ETA: 0s - loss: 0.6597 - accuracy: 0.5959
Epoch 00027: saving model to checkpoints\weights.27.hdf5
487/804 [=================>............] - ETA: 0s - loss: 0.6612 - accuracy: 0.5925
Epoch 00027: saving model to checkpoints\weights.27.hdf5
552/804 [===================>..........] - ETA: 0s - loss: 0.6612 - accuracy: 0.5932
Epoch 00027: saving model to checkpoints\weights.27.hdf5
690/804 [========================>.....] - ETA: 0s - loss: 0.6618 - accuracy: 0.5937
Epoch 00027: saving model to checkpoints\weights.27.hdf5
753/804 [===========================>..] - ETA: 0s - loss: 0.6614 - accuracy: 0.5942
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/80

...

804/804 [==============================] - 1s 721us/step - loss: 0.6613 - accuracy: 0.5950
Epoch 73/100
  1/804 [..............................] - ETA: 0s - loss: 0.6910 - accuracy: 0.4688
Epoch 00073: saving model to checkpoints\weights.73.hdf5
 66/804 [=>............................] - ETA: 0s - loss: 0.6535 - accuracy: 0.6146
Epoch 00073: saving model to checkpoints\weights.73.hdf5
211/804 [======>.......................] - ETA: 0s - loss: 0.6639 - accuracy: 0.5923
Epoch 00073: saving model to checkpoints\weights.73.hdf5
277/804 [=========>....................] - ETA: 0s - loss: 0.6622 - accuracy: 0.5950
Epoch 00073: saving model to checkpoints\weights.73.hdf5
342/804 [===========>..................] - ETA: 0s - loss: 0.6619 - accuracy: 0.5947
Epoch 00073: saving model to checkpoints\weights.73.hdf5
490/804 [=================>............] - ETA: 0s - loss: 0.6620 - accuracy: 0.5934
Epoch 00073: saving model to checkpoints\weights.73.hdf5
555/804 [===================>..........] - E

369/804 [============>.................] - ETA: 0s - loss: 0.6607 - accuracy: 0.5965
Epoch 00079: saving model to checkpoints\weights.79.hdf5
431/804 [===============>..............] - ETA: 0s - loss: 0.6609 - accuracy: 0.5955
Epoch 00079: saving model to checkpoints\weights.79.hdf5
574/804 [====================>.........] - ETA: 0s - loss: 0.6615 - accuracy: 0.5936
Epoch 00079: saving model to checkpoints\weights.79.hdf5
640/804 [======================>.......] - ETA: 0s - loss: 0.6613 - accuracy: 0.5944
Epoch 00079: saving model to checkpoints\weights.79.hdf5
783/804 [============================>.] - ETA: 0s - loss: 0.6615 - accuracy: 0.5943
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 721us/step - loss: 0.6611 - accuracy: 0.5953
Epoch 80/100
 79/804 [=>............................] - ETA: 0s - loss: 0.6621 - accuracy: 0.5763
Epoch 00080: saving model to checkpoints\weights.80.hdf5
146/804 [====>.........................] - E

804/804 [==============================] - 1s 726us/step - loss: 0.6613 - accuracy: 0.5950
Epoch 86/100
  1/804 [..............................] - ETA: 0s - loss: 0.6520 - accuracy: 0.6250
Epoch 00086: saving model to checkpoints\weights.86.hdf5
146/804 [====>.........................] - ETA: 0s - loss: 0.6541 - accuracy: 0.6036
Epoch 00086: saving model to checkpoints\weights.86.hdf5
213/804 [======>.......................] - ETA: 0s - loss: 0.6598 - accuracy: 0.5940
Epoch 00086: saving model to checkpoints\weights.86.hdf5
357/804 [============>.................] - ETA: 0s - loss: 0.6599 - accuracy: 0.5955
Epoch 00086: saving model to checkpoints\weights.86.hdf5
423/804 [==============>...............] - ETA: 0s - loss: 0.6601 - accuracy: 0.5955
Epoch 00086: saving model to checkpoints\weights.86.hdf5
488/804 [=================>............] - ETA: 0s - loss: 0.6597 - accuracy: 0.5973
Epoch 00086: saving model to checkpoints\weights.86.hdf5
638/804 [======================>.......] - E

422/804 [==============>...............] - ETA: 0s - loss: 0.6599 - accuracy: 0.5990
Epoch 00092: saving model to checkpoints\weights.92.hdf5
488/804 [=================>............] - ETA: 0s - loss: 0.6602 - accuracy: 0.5971
Epoch 00092: saving model to checkpoints\weights.92.hdf5
632/804 [======================>.......] - ETA: 0s - loss: 0.6604 - accuracy: 0.5954
Epoch 00092: saving model to checkpoints\weights.92.hdf5
698/804 [=========================>....] - ETA: 0s - loss: 0.6601 - accuracy: 0.5968
Epoch 00092: saving model to checkpoints\weights.92.hdf5
804/804 [==============================] - 1s 720us/step - loss: 0.6607 - accuracy: 0.5948
Epoch 93/100
  1/804 [..............................] - ETA: 0s - loss: 0.5744 - accuracy: 0.6875
Epoch 00093: saving model to checkpoints\weights.93.hdf5
 67/804 [=>............................] - ETA: 0s - loss: 0.6624 - accuracy: 0.5877
Epoch 00093: saving model to checkpoints\weights.93.hdf5
212/804 [======>.......................] - E

  1/804 [..............................] - ETA: 0s - loss: 0.6396 - accuracy: 0.6562
Epoch 00099: saving model to checkpoints\weights.99.hdf5
 67/804 [=>............................] - ETA: 0s - loss: 0.6539 - accuracy: 0.6091
Epoch 00099: saving model to checkpoints\weights.99.hdf5
132/804 [===>..........................] - ETA: 0s - loss: 0.6575 - accuracy: 0.6035
Epoch 00099: saving model to checkpoints\weights.99.hdf5
287/804 [=========>....................] - ETA: 0s - loss: 0.6594 - accuracy: 0.5979
Epoch 00099: saving model to checkpoints\weights.99.hdf5
353/804 [============>.................] - ETA: 0s - loss: 0.6601 - accuracy: 0.5973
Epoch 00099: saving model to checkpoints\weights.99.hdf5
496/804 [=================>............] - ETA: 0s - loss: 0.6599 - accuracy: 0.5987
Epoch 00099: saving model to checkpoints\weights.99.hdf5
563/804 [====================>.........] - ETA: 0s - loss: 0.6603 - accuracy: 0.5972
Epoch 00099: saving model to checkpoints\weights.99.hdf5
708/80

In [537]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 60.5392 - accuracy: 0.5448
Loss: 60.53921127319336, Accuracy: 0.5447970032691956


In [538]:
nn.save("AlphabetSoupCharity_Optimization.h5")